In [1]:
# import all required packages
import spacy
import re
import pandas as pd
import numpy as np
import gensim
from gensim.utils import simple_preprocess 
import gensim.corpora as corpora
from pprint import pprint

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Reading loading/data
data = pd.read_csv("https://raw.githubusercontent.com/SandieIJ/Capstone/master/data/sandra_csv_results-20190723-155508.csv")

# Sample of the output
data.head(10)

,name,description
0,Legend Fire Squad survival: Free Fire Battlegr...,Ready to play an amazing and exciting best sho...
1,Ambulance Game,You must be a fan of the driving games. We ass...
2,Beam Drive NG Death Stair Car Crash Simulator,Beam Drive NG Death Stair Car Crash Accidents ...
3,Kelime İncileri,Yeni Kelime Bulmaca Oyununuz! Kelime Arama ve ...
4,Word Blocks,Word Blocks is a new kind of word search puzzl...
5,Free Fire Commando - Counter Attack FPS 2019,Free Fire Commando - Counter Attack FPS 2019 i...
6,Fall Race 3D,The most exciting sky race!Run through the sky...
7,Math School Game Basic: Crazy Principal,Your school principal went crazy and locked yo...
8,Jump Cube,"Jump Cube is an addictive game, tap the right ..."
9,Tien Len Offline,Một tựa game cũng như cách chơi ko thể quen th...


In [4]:
type(data)

pandas.core.frame.DataFrame

In [5]:
# convert the descriptions from a data frame column into a list
descriptions = data.description.values.tolist()

# sample of the output
print(descriptions[10])

Brick Breaker 3D is a single-tap hyper casual game that will keep you hooked for hours!Hold the screen to aim, swipe the ball to the brick and break all the bricks easily!The game features unlimited levels and 20 beautiful color balls.


In [6]:
no_new_lines = [re.sub('\s+', ' ', sent) for sent in descriptions] 

#Remove non letter characters
non_letters = [re.sub('[^a-zA-Z]', ' ', no_new_line) \
                    for no_new_line in no_new_lines]

# Remove distracting single quotes
no_quotes = [re.sub("\'", '', non_letter) for \
              non_letter in non_letters]

In [7]:
#break down sentences into words
def sent_to_words(sentences): 
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(no_quotes))

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 
trigram = gensim.models.Phrases(bigram[no_quotes], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram) 

trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Form Bigrams
data_words_bigrams = make_bigrams(data_words)